In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

# Add /src modules to path
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Import modules
from src.utils import DbConnector, haversine_np
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from collections import Counter


In [3]:
# Connect to MySQL database

connector = DbConnector()
db = connector.db

You are connected to the database: mongodb
-----------------------------------------------



----

### Task 1
How many users, activities and trackpoints are there in the dataset?

In [4]:
collection = db["ActivityCollection"]

users_count = db.UserCollection.count_documents({})
activities_count = db.ActivityCollection.count_documents({})
trackpoints_count = db.ActivityCollection.aggregate(
    [{"$unwind": "$track_points"}, {"$count": "trackpoint_count"}]
)
trackpoints_count = next(trackpoints_count, {}).get("trackpoint_count", 0)


print(f"Total Users: {users_count}")
print(f"Total Activities: {activities_count}")
print(f"Total Trackpoints: {trackpoints_count}")

Total Users: 182
Total Activities: 15641
Total Trackpoints: 8501885


### Task 2
Find the average number of activities per user

In [5]:
average_activities_per_user = activities_count / users_count
print(f"Average Activities/User: {average_activities_per_user:.2f}")

Average Activities/User: 85.94


### Task 3
Find the top 20 users with the highest number of activities. 

In [6]:
users_activities_count = list(
    db["ActivityCollection"].aggregate(
        [
            {"$group": {"_id": "$user", "activity_count": {"$sum": 1}}},
            {"$sort": {"activity_count": -1}},
            {"$limit": 20},
        ]
    )
)

print("Top 20 Users with Most Activities:")

df = pd.DataFrame(users_activities_count)
df = df.rename(columns={'_id':'user_id'})
df

Top 20 Users with Most Activities:


,user_id,activity_count
0,085,1089
1,153,977
2,068,920
3,128,876
4,167,810
5,025,715
6,062,538
7,126,420
8,084,411
9,010,402


### Task 4
Find all users who have taken a taxi. 

In [18]:
users_taken_taxi = db["ActivityCollection"].distinct(
    "user", {"transportation_mode": "taxi"}
)

print(f"{str(len(users_taken_taxi))} users have taken a taxi:")

df = pd.DataFrame(users_taken_taxi)
df = df.rename(columns={0:'user_id'})
df.head(10)

29 users have taken a taxi:


,user_id
0,010
1,021
2,052
3,056
4,058
5,062
6,065
7,068
8,075
9,078


### Task 5
Find all types of transportation modes and count how many activities that are
tagged with these transportation mode labels. Do not count the rows where
the mode is null.

In [58]:
transportation_modes = list(
    db["ActivityCollection"].aggregate(
        [
            {"$group": {"_id": "$transportation_mode", "count": {"$sum": 1}}},
            {"$match": {"_id": {"$ne": None}}},
        ]
    )
)

print("Transportation Modes and Their Counts:")

df = pd.DataFrame(transportation_modes)
df = df.sort_values('count', ascending=False)
df = df.rename(columns={'_id':'transportation_mode', 'count':'tagged_count'})
df = df.reset_index(drop=True)
df

Transportation Modes and Their Counts:


,transportation_mode,tagged_count
0,walk,3927
1,bus,1820
2,bike,1519
3,car,751
4,subway,613
5,taxi,512
6,train,134
7,airplane,13
8,boat,7
9,run,4


### Task 6

#### a) Find the year with the most activities. 

In [59]:
year_most_activities = list(
    db.ActivityCollection.aggregate(
        [
            {"$project": {"year": {"$year": "$start_date_time"}}},
            {"$group": {"_id": "$year", "activity_count": {"$sum": 1}}},
            {"$sort": {"activity_count": -1}},
            {"$limit": 1},
        ]
    )
)[0]

print(f"Year with most activities: {year_most_activities['_id']}")
print(f"This year had {year_most_activities['activity_count']} activities")

Year with most activities: 2008
This year had 7861 activities


#### b) Is this also the year with most recorded hours?

This aggregation pipeline first calculates the number of hours for each activity by subtracting the start_date_time from the end_date_time (resulting in a duration in milliseconds) and then dividing by the number of milliseconds in an hour (3,600,000). It then groups the data by year and sums the total hours, sorts by total hours in descending order, and limits the results to the year with the highest number of hours.

In [60]:
year_most_hours = list(
    db.ActivityCollection.aggregate(
        [
            {
                "$project": {
                    "year": {"$year": "$start_date_time"},
                    "hours": {
                        "$divide": [
                            {"$subtract": ["$end_date_time", "$start_date_time"]},
                            3600000, 
                        ]
                    },
                }
            },
            {"$group": {"_id": "$year", "total_hours": {"$sum": "$hours"}}},
            {"$sort": {"total_hours": -1}},
            {"$limit": 1},
        ]
    )
)[0]

print(
    f"Year with most recorded hours: {year_most_hours['_id']}, Hours: {year_most_hours['total_hours']:.2f}"
)

Year with most recorded hours: 2009, Hours: 9765.33


### Task 7
Find the total distance (in km) walked in 2008, by user with id=112.

In [61]:
user_id = "112"
activities_for_user = db.ActivityCollection.find({
    "user": user_id,
    "transportation_mode": "walk",
    "start_date_time": {"$gte": datetime(2008, 1, 1), "$lt": datetime(2009, 1, 1)}
})

total_distance = 0.0

for activity in activities_for_user:
    track_points = activity["track_points"]
    
    lats = np.array([point["latitude"] for point in track_points])
    longs = np.array([point["longitude"] for point in track_points])
    
    total_distance += np.sum(haversine_np(longs[:-1], lats[:-1], longs[1:], lats[1:]))

print(f"Total distance walked by user {user_id} in 2008: {total_distance:.2f} km")


Total distance walked by user 112 in 2008: 223.15 km


### Task 8
Find the top 20 users who have gained the most altitude meters.
- Output should be a field with (id, total meters gained per user).
- Remember that some altitude-values are invalid

In [68]:
def calculate_altitude_gain(track_points):
    return sum(
        max(0, b["altitude"] - a["altitude"])
        for a, b in zip(track_points, track_points[1:])
    )


activity_collection = db["ActivityCollection"]

user_altitude_gains = {}

for activity in activity_collection.find():
    user = activity["user"]
    altitude_gain = calculate_altitude_gain(activity["track_points"])

    # Convert altitude from feet to meters
    altitude_gain *= 0.3048

    if user in user_altitude_gains:
        user_altitude_gains[user] += altitude_gain
    else:
        user_altitude_gains[user] = altitude_gain

sorted_users = sorted(user_altitude_gains.items(), key=lambda x: x[1], reverse=True)

pd.options.display.float_format = "{:.0f}".format

df = pd.DataFrame(sorted_users)
df = df.rename(columns={0: "user_id", 1: "meters_altitude_gain"})
df = df.sort_values("meters_altitude_gain", ascending=False)
df.head(20)

,user_id,meters_altitude_gain
0,004,332036
1,128,266359
2,085,259790
3,041,240769
4,003,233664
5,030,175680
6,062,152512
7,039,146704
8,084,132726
9,167,129899


### Task 9
Find all users who have invalid activities, and the number of invalid activities
per user.
- An invalid activity is defined as an activity with consecutive trackpoints
where the timestamps deviate with at least 5 minutes. 

In [79]:
def has_invalid_activity(track_points):
    for i in range(1, len(track_points)):
        if track_points[i]["date_time"] - track_points[i - 1]["date_time"] >= timedelta(
            minutes=5
        ):
            return True
    return False


activity_collection = db["ActivityCollection"]

user_invalid_activity_counts = {}

for activity in activity_collection.find():
    user = activity["user"]
    if has_invalid_activity(activity["track_points"]):
        if user in user_invalid_activity_counts:
            user_invalid_activity_counts[user] += 1
        else:
            user_invalid_activity_counts[user] = 1


df = pd.DataFrame(user_invalid_activity_counts.items(), columns=["user_id", "invalid_activities_count"])
df = df[df.invalid_activities_count > 0]
df.sort_values("user_id").reset_index(drop=True)

,user_id,invalid_activities_count
0,000,101
1,001,45
2,002,98
3,003,179
4,004,219
...,...,...
157,175,5
158,176,8
159,179,53
160,180,2


### Task 10
Find the users who have tracked an activity in the Forbidden City of Beijing.
- In this question you can consider the Forbidden City to have
coordinates that correspond to: lat 39.916, lon 116.397.

In [65]:
coordinates = {"latitude": 39.916, "longitude": 116.397}

pipeline = [
    {"$unwind": "$track_points"},
    {
        "$match": {
            "track_points.latitude": {
                "$gte": coordinates["latitude"] - 0.001,
                "$lte": coordinates["latitude"] + 0.001,
            },
            "track_points.longitude": {
                "$gte": coordinates["longitude"] - 0.001,
                "$lte": coordinates["longitude"] + 0.001,
            },
        }
    },
    {"$group": {"_id": "$user"}},
]

output = db.ActivityCollection.aggregate(pipeline)
output = [user["_id"] for user in output]
output

['018', '019', '004', '131', '153']

### Task 11
Find all users who have registered transportation_mode and their most used
transportation_mode. 
- The answer should be on format (user_id,
most_used_transportation_mode) sorted on user_id.
- Some users may have the same number of activities tagged with e.g.
walk and car. In this case it is up to you to decide which transportation
mode to include in your answer (choose one).
- Do not count the rows where the mode is null.

In [19]:
users_most_used_mode = {}
all_users = db.UserCollection.find({})

for user in all_users:
    user_id = user["_id"]
    user_activities = list(
        db.ActivityCollection.find(
            {"user": user_id, "transportation_mode": {"$ne": None}}
        )
    )
    modes = [activity["transportation_mode"] for activity in user_activities]

    modes_count = Counter(modes)
    if modes_count:
        users_most_used_mode[user_id] = modes_count.most_common(1)[0][0]

print("Users and their most used transportation mode:")

df = pd.DataFrame(users_most_used_mode.items())
df = df.rename(columns={0: "User", 1: "Most used mode"})
df = df.sort_values("User")
df = df.reset_index(drop=True)
df

Users and their most used transportation mode:


,User,Most used mode
0,010,walk
1,020,bike
2,021,car
3,052,bus
4,053,walk
...,...,...
59,167,walk
60,170,walk
61,174,car
62,175,walk


----

In [20]:
connector.close_connection()


-----------------------------------------------
Connection to mongodb-db is closed
